<a href="https://colab.research.google.com/github/JS0501/ESAA_OB/blob/main/ESAA1017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 07. 행렬 분해를 이용한 잠재 요인 협업 필터링 실습

In [8]:
import numpy as np

def get_rmse(R, P, Q, non_zeros):
    error = 0
    for i, j, r in non_zeros:
        pred = np.dot(P[i, :], Q[j, :].T)
        error += pow(r - pred, 2)
    rmse = np.sqrt(error / len(non_zeros))
    return rmse

In [9]:
def matrix_factorization(R,K,steps=200,learning_rate=0.01,r_lambda=0.01):
  num_users, num_items = R.shape
  # P와 Q 매트릭스의 크기를 지정하고 정규 분포를 가진 랜덤한 값으로 입력합니다.
  np.random.seed(1)
  P = np.random.normal(scale=1./K, size=(num_users, K))
  Q = np.random.normal(scale=1./K, size=(num_items, K))

  # R>0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장.
  non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]

  # SGD 기법으로 P와 Q 매트릭스를 계속 업데이트.
  for step in range(steps):
    for i,j,r in non_zeros:
      # 실제 값과 예측 값의 차이인 오류 값 구함
      eij = r - np.dot(P[i,:],Q[j,:].T)
      # Regularization을 반영한 SGD 업데이트 공식 적용
      P[i,:] = P[i,:] + learning_rate*(eij*Q[j,:]-r_lambda*P[i,:])
      Q[j,:] = Q[j,:] + learning_rate*(eij*P[i,:]-r_lambda*Q[j,:])

    rmse = get_rmse(R,P,Q,non_zeros)
    if (step%10) == 0:
      print('### iteration step:',step,"rmse:",rmse)

  return P,Q

영화 평점 행렬 데이터를 새롭게 DataFrame으로 로딩한 뒤 다시 사용자-아이템 평점 행렬로 만든다.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

movies = pd.read_csv('/content/drive/MyDrive/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv')
ratings = ratings[['userId','movieId','rating']]
ratings_matrix = ratings.pivot_table('rating',index='userId',columns='movieId')

# title 칼럼을 얻기 위해 movies와 조인 수행
rating_movies = pd.merge(ratings,movies,on='movieId')

# columns='title'로 title 칼럼으로 pivot 수행.
ratings_matrix = rating_movies.pivot_table('rating',index='userId',columns='title')

다시 만들어진 사용자-아이템 평점 행렬을 matrix_factorization() 함수를 이용해 행렬 분해한다. 잠재 요인차원 K는 50, 학습률과 L2 Regularization 계수는 모두 0.01로 설정하고 수행한다.

In [10]:
P,Q = matrix_factorization(ratings_matrix.values,K=50,steps=200,learning_rate=0.01,r_lambda=0.01)
pred_matrix = np.dot(P,Q.T)

### iteration step: 0 rmse: 2.9023619751337115
### iteration step: 10 rmse: 0.7335768591017939
### iteration step: 20 rmse: 0.5115539026853438
### iteration step: 30 rmse: 0.37261628282537734
### iteration step: 40 rmse: 0.29608182991810145
### iteration step: 50 rmse: 0.2520353192341621
### iteration step: 60 rmse: 0.22487503275269882
### iteration step: 70 rmse: 0.20685455302331512
### iteration step: 80 rmse: 0.19413418783028674
### iteration step: 90 rmse: 0.1847008200272031
### iteration step: 100 rmse: 0.17742927527209082
### iteration step: 110 rmse: 0.17165226964707506
### iteration step: 120 rmse: 0.16695181946871496
### iteration step: 130 rmse: 0.16305292191997453
### iteration step: 140 rmse: 0.159766919296796
### iteration step: 150 rmse: 0.15695986999457337
### iteration step: 160 rmse: 0.15453398186715442
### iteration step: 170 rmse: 0.1524161855107769
### iteration step: 180 rmse: 0.1505508073962834
### iteration step: 190 rmse: 0.14889470913232075


더 쉽게 영화 아이템 칼럼을 이해하기 위해 반환된 예측 사용자-아이템 평점 행렬을 영화 타이틀을 칼럼명으로 가지는 DataFrame으로 변경한다.

In [12]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= ratings_matrix.index, columns = ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941


이렇게 만들어짐 예측 사용자-아이템 평점 행렬 정보를 이용해 개인화된 영화 추천을 한다.

In [15]:
def get_unseen_movies(ratings_matrix,userId):
  # userId로 입력받은 사용자의 모든 영화 정보를 추출해 Series로 반환함.
  # 반환된 user_ratnig은 영화명(title)을 인덱스로 가지는 Series 객체임.
  user_rating = ratings_matrix.loc[userId,:]

  # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 인덱스를 추출해 list 객체로 만듦.
  already_seen = user_rating[user_rating>0].index.tolist()

  # 모든 영화명을 list 객체로 만듦.
  movies_list = ratings_matrix.columns.tolist()

  # list comprehension으로 already_seen에 해당하는 영화는 movies_list에서 제외함.
  unseen_list = [movie for movie in movies_list if movie not in already_seen]

  return unseen_list

def recomm_movie_by_userid(pred_df,userId,unseed_list,top_n=10):
  # 예측 평점 DataFrame에서 사용자id 인덱스와 unseen_list로 들어온 영화명 칼럼을 추출해
  # 가장 예측 평점이 높은 순으로 정렬함.
  recomm_movies = pred_df.loc[userId,unseen_list].sort_values(ascending=False)[:top_n]
  return recomm_movies

# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix,9)

# 아이템 기반의 최근접 이웃 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix,9,unseen_list,top_n=10)

# 평점 데이터를 DataFrame으로 생성
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690


In [16]:
# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 잠재 요인 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이터를 DataFrame으로 생성.
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690


앞 절의 아이템 기반 협업 필터링 결과와 추천된 영화가 많이 다르다.

### 08. 파이썬 추천 시스템 패키지 - Surprise

In [1]:
!pip install numpy==1.26.4 --force-reinstall --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 199.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-

In [17]:
! pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2611194 sha256=52b6a5e69810e3ab32299b4603a81234944bf5751945d044b499e8ab2df93125
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


Surprise 패키지 장점

- 다양한 추천 알고리즘. 예를 들어 사용자 또는 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반의 잠재 요인 협업 필터링을 쉽게 적용해 추천 시스템을 구축할 수 있다.

- Surprise의 핵심 API는 사이킷런의 핵심 API와 유사한 API명으로 작성되었다. 예를 들어 fit(), predict() API로 추천 데이터 학습과 예측, train_test_split()으로 추천 학습 데이터 세트와 예측 데이터 세트 분리, cross_validate(), GridSearchCV 클래스를 통해 추천 시스템을 위한 모델 셀렉션, 평가, 하이퍼 파라미터 튜닝 등의 기능을 제공한다.

**Surprise를 이용한 추천 시스템 구축**

In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

Surprise에서 데이터 로딩은 Dataset 클래스를 이용해서만 가능하다.

Surprise는 Movie Lens 데이터 세트의 사용자-영화 평점 데이터 포맷과 같이 userId, movieId, rating과 같은 주요 데이터가 row 형태로 되어 있는 포맷의 데이터만 처리한다.

In [2]:
data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data,test_size=.25,random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


SVD로 잠재 요인 협업 필터링을 수행한다.

적용하는 데이터 세트는 train_test_split()으로 분리된 학습 데이터 세트이다.

먼저 algo=SVD()와 같이 알고리즘 객체를 생성한다.

이 알고리즘 객체에 fit(학습 데이터 세트)을 수행해 학습 데이터 세트 기반으로 추천 알고리즘을 학습한다.

In [3]:
algo = SVD(random_state=0)
algo.fit(trainset)

학습된 추천 알고리즘을 기반으로 테스트 데이터 세트에 대해 추천을수행한다.

In [4]:
predictions = algo.test(testset)
print('prediction type:',type(predictions),'size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type: <class 'list'> size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

'was_impossible'이 True이면 예측값을 생성할 수 없는 데이터라는 의미이다. 여기서는 모두 False이다.

In [5]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

이번에는 Surprise 패키지의 다른 추천 예측 메서드인 predict()를 이용해 추천 예측을 해본다.

predict()는 개별 사용자의 아이템에 대한 추천 평점을 예측한다

인자로 개별 사용자 아이디, 아이템 아이디를 입력하면 추천 예측 평점을 포함한 정보를 반환한다

In [6]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함.
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


결과처럼 predict()는 개별 사용자와 아이템 정보를 입력하면 추천 예측 평점을 est로 반환한다.

테스트 데이터 세트를 이용해 추천 예측 평점과 실제 평점과의 차이를 평가한다.

Surprise의 accuracy 모듈은 RMSE, MSE 등의 방법으로 추천 시스템의 성능 평가 정보를 제공한다.

In [7]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

**Surprise 주요 모듈 소개**

**Dataset**

Surprise는 user_id, item_id, rating 데이터가 로우 레벨로 된 데이터 세트에만 적용할 수 있다.

**OS 파일 데이터를 Surprise 데이터 세트로 로딩**

In [8]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv')
# ratings_noh.csv 파일로 업로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성.
ratings.to_csv('/content/drive/MyDrive/ratings_noh.csv',index=False,header=False)

In [9]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp',sep=',',rating_scale=(0.5,5))
data = Dataset.load_from_file('/content/drive/MyDrive/ratings_noh.csv',reader=reader)

Surprise 데이터 세트는 기본적으로 무비렌즈 데이터 형식을 따르므로 무비렌즈 데이터 형식이 아닌 다른 OS 파일의 경우 Reader 클래스를 먼저 설정해야 한다.

Reader 클래스의 주요 생성 파라미터

- line_format(string): 칼럼을 순서대로 나열한다. 입력된 문자열을 공백으로 분리해 칼럼으로 인식한다.

- sep(char): 칼럼을 분리하는 분리자이며, 디폴트는 '/t'이다. 판다스 DataFrame에서 입력받을 경우에는 기재할 필요가 없다.

- rating_scale (tuple,optional): 평점 값의 최소~최대 평점을 설정한다.

In [10]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)
 # 수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)
# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

**판다스 DataFrame에서 Surprise 데이터 세트로 로딩**

In [12]:
import pandas as pd
from surprise import Reader, Dataset
ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))
 # ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

**베이스라인 평점**

세미나를 평가해달라는 요구에 어떤 사람은 엄격한 잣대를 가지고 평가하고, 어떤 사람은 좋은 평가 점수를 주는 경우도 있다.

영화나 상품의 평가도 각 개인의 성향에 따라 같은 아이템이더라도 평가가 달라질 수 있다.

싫은 소리를 별로 안 하는 사람의경우 전반적으로 평가에 후한 경향이 있다.

반면 다른 이를 생각해서라도 냉정한 평가를 해야 한다고 생각하는 사람도 있다.

이러한 개인의 성향을 반영해 아이템 평가에 편향성 요소를 반영하여 평점을 부과하는 것을 베이스라인 평점이라고 한다.

보통 베이스라인 평점은 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 공식으로 계산된다.

**교차 검증과 하이퍼파라미터 튜닝**

Surprise는 교차 검증과 하이퍼 파라미터 튜닝을 위해 사이킷런과 유사한 cross_validate()와 GridSearchCV 클래스를 제공한다.

In [14]:
from surprise.model_selection import cross_validate
 # 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(random_state=0)
cross_validate(algo, data, measures=[ 'RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8714  0.8755  0.8772  0.8725  0.8693  0.8732  0.0028  
MAE (testset)     0.6707  0.6720  0.6722  0.6709  0.6690  0.6710  0.0011  
Fit time          1.54    1.54    1.51    2.14    1.99    1.74    0.26    
Test time         0.29    0.11    0.24    0.43    0.20    0.26    0.10    


{'test_rmse': array([0.87138793, 0.87547451, 0.87717489, 0.87246528, 0.86927743]),
 'test_mae': array([0.67071706, 0.67199566, 0.67222524, 0.67094595, 0.66902824]),
 'fit_time': (1.5446667671203613,
  1.5438480377197266,
  1.5060985088348389,
  2.136918544769287,
  1.9861047267913818),
 'test_time': (0.28963470458984375,
  0.11410069465637207,
  0.24448418617248535,
  0.43131256103515625,
  0.20430922508239746)}

cross_validate()는 위의 출력 결과와 같이 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치를 함께 보여준다.

In [15]:
from surprise.model_selection import GridSearchCV
 # 최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }
 # CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)
 # 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.876019261105403
{'n_epochs': 20, 'n_factors': 50}


**Surprise를 이용한 개인화 영화 추천 시스템 구축**

이번 예제에서는 ratings.csv 데이터를 학습 데이터와 테스트 데이터로 분리하지 않고 전체를 학습 데이터로 사용한다.

그런데 Surprise는 데이터 세트를 train_test_split()을 이용해 내부에서 사용하는 TrainSet 클래스 객체로 변환하지 않으면 fit()을 통해 학습할 수 없다.

따라서 데이터 세트를 그대로 fit()에 적용한 다음 코드는 오류를 일으킨다.

In [16]:
from surprise.dataset import DatasetAutoFolds
from surprise import Reader, SVD
import pandas as pd

# Load the ratings data
ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv')

# Define the reader
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))

# Save ratings DataFrame to a temporary CSV file without header for DatasetAutoFolds
ratings.to_csv('/tmp/ratings_noh.csv', index=False, header=False)

# Create DatasetAutoFolds from the CSV file
data_folds = DatasetAutoFolds(ratings_file='/tmp/ratings_noh.csv', reader=reader)

# Build the full training set
trainset = data_folds.build_full_trainset()

# Initialize and train the SVD algorithm
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)

AttributeError: 'DatasetAutoFolds' object has no attribute 'n_users'

데이터 세트 전체를 학습 데이터로 사용하려면 DatasetAutoFolds 클래스를 이용한다.

In [18]:
from surprise.dataset import DatasetAutoFolds
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
 # DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성.
data_folds = DatasetAutoFolds(ratings_file='/content/drive/MyDrive/ratings_noh.csv', reader=reader)
 # 전체 데이터를 학습 데이터로 생성함.
trainset = data_folds.build_full_trainset()

DatasetAutoFolds의 build_full_trainset() 메서드를 이용해 생성된 학습 데이터를 기반으로 학습을 수행한다.

그리고 이후에 특정 사용자에 영화를 추천하기 위해 아직 보지 않은 영화 목록을 확인한다.

먼저 SVD를 이용해 학습을 수행한다.

In [19]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [21]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('/content/drive/MyDrive/movies.csv')
# userld=9의 movield 데이터를 추출해 movield=42 데이터가 있는지 확인.
movields = ratings[ratings[ 'userId' ]==9] [ 'movieId' ]
if movields[movields==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
print(movies[movies['movieId' ]==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [22]:
uid = str(9)
iid = str(42)
pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [23]:
def get_unseen_surprise(ratings, movies, userId):
    # 입력값으로 들어온 userId가 평점을 매긴 영화 목록
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

    # 모든 영화의 movieId 목록
    total_movies = movies['movieId'].tolist()

    # 이미 평점 매긴 영화 제외
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]

    print('평점 매긴 영화 수:', len(seen_movies),
          '추천 대상 영화 수:', len(unseen_movies),
          '전체 영화 수:', len(total_movies))

    return unseen_movies

# 예시: userId가 9인 경우
unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [24]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, movies, top_n=10):
    # 알고리즘 객체의 predict 메서드를 평점이 없는 영화에 반복 수행
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    # est 값을 기준으로 정렬
    def sortkey_est(pred):
        return pred.est

    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]

    # top_n으로 추출된 영화 정보
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title'].tolist()

    top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    return top_movie_preds

# unseen_movies 생성
unseen_movies = get_unseen_surprise(ratings, movies, 9)

# 추천 수행
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, movies, top_n=10)

# 결과 출력
print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
    print(top_movie[1], ":", round(top_movie[2], 2))

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.31
Star Wars: Episode IV - A New Hope (1977) : 4.28
Pulp Fiction (1994) : 4.28
Silence of the Lambs, The (1991) : 4.23
Godfather, The (1972) : 4.19
Streetcar Named Desire, A (1951) : 4.15
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.12
Star Wars: Episode VI - Return of the Jedi (1983) : 4.11
Goodfellas (1990) : 4.08
Glory (1989) : 4.08


추천 시스템은 기업 애플리케이션에서 매우 중요한 위치를 차지한다. 특히 온라인 스토어의 경우 뛰어난 추천 시스템은 매출 향상으로 이어지는 많은 사례로 인해 많은 기업들이 추천 시스템의 예측 성능을 향상시키기 위해 알고리즘과 데이터 수집에 노력을 기울이고 있다.

콘텐츠 기반 필터링은 아이템(상품,영화,서비스 등)을 구성하는 여러 가지 콘텐츠 중 사용자가 좋아하는 콘텐츠를 필터링하여 이에 맞는 아이템을 추천하는 방식이다.

협업 필터링은 최근접 이웃 협업 필터링과 잠재 요인 협업 필터링으로 나뉜다.

최근접 이웃 협업 필터링은 다시 사용자 기반과 아이템 기반으로 나뉘며, 이중 아이템 기반이 더 많이 사용된다.

아이템 기반 최근접 이웃 방식은 특정 아이템과 가장 근접하게 유사한 다른 아이템들을 추천하는 방식이다. 이 유사도의 기준이 되는 것은 사용자들의 아이템에 대한 평가를 벡터화한 값이다.

잠재 요인 협업 필터링은 사용자-아이템 평점 행렬 데이터에 숨어 있는 잠재 요인을 추출하여 사용자가 아직 평점을 매기지 않은 아이템에 대한 평점을 예측하여 이를 추천에 반영하는 방식이다.